In [14]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [15]:
season_list = ['1718', '1819', '1920', '2021', '2122', '2223', '2324', '2425']
league_eng = 'E0'


df_list_eng = []

for season in season_list:
    data_url = f'https://www.football-data.co.uk/mmz4281/{season}/{league_eng}.csv'
    df = pd.read_csv(data_url)
    df_list_eng.append(df)

main_df = pd.concat(df_list_eng)

main_df['Date'] = pd.to_datetime(main_df['Date'], format='%d/%m/%Y')
main_df = main_df.sort_values(by='Date')

df = main_df
df = df[::-1]

In [16]:
df_swe = pd.read_csv('https://www.football-data.co.uk/new/SWE.csv')

df_swe['Date'] = pd.to_datetime(df_swe['Date'], format='%d/%m/%Y')
df_swe= df_swe.sort_values(by='Date')
df_swe = df_swe[::-1]

df_swe.head()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PSCH,PSCD,PSCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,BFECH,BFECD,BFECA
3167,Sweden,Allsvenskan,2025,2025-04-14,18:10,AIK,Malmo FF,0,0,D,4.30,3.36,1.98,4.42,3.45,2.05,4.16,3.29,1.95,4.50,3.45,2.02
3166,Sweden,Allsvenskan,2025,2025-04-14,18:00,Halmstad,Oster,1,0,H,3.09,3.24,2.49,3.10,3.30,2.51,2.97,3.18,2.44,3.10,3.25,2.60
3165,Sweden,Allsvenskan,2025,2025-04-13,15:30,Sirius,Brommapojkarna,0,3,A,2.25,3.59,3.22,2.28,3.59,3.22,2.22,3.47,3.12,2.34,3.65,3.30
3162,Sweden,Allsvenskan,2025,2025-04-13,13:00,Hammarby,Djurgarden,2,0,H,2.33,3.30,3.33,2.36,3.40,3.60,2.25,3.25,3.26,2.34,3.35,3.50
3164,Sweden,Allsvenskan,2025,2025-04-13,15:30,GAIS,Degerfors,2,0,H,1.83,3.62,4.66,1.85,3.75,4.66,1.82,3.56,4.39,1.90,3.60,4.90


In [17]:
def get_teams_result(df):
    resultat = {}

    teams = set(df["HomeTeam"]).union(set(df["AwayTeam"]))

    for team in teams:
        resultat[team] = {}
        
    for _, rad in df.iterrows():
        hemma, borta = rad["HomeTeam"], rad["AwayTeam"]
        mål_a, mål_b = rad["FTHG"], rad["FTAG"]

        if hemma not in resultat:
            resultat[hemma] = {}
        if borta not in resultat:
            resultat[borta] = {}

        if borta not in resultat[hemma]:
            resultat[hemma][borta] = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}
        if hemma not in resultat[borta]:
            resultat[borta][hemma] = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}

        if mål_a > mål_b: 
            resultat[hemma][borta]["Vinster"] += 1
            resultat[borta][hemma]["Förluster"] += 1
        elif mål_a < mål_b: 
            resultat[borta][hemma]["Vinster"] += 1
            resultat[hemma][borta]["Förluster"] += 1
        else:
            resultat[hemma][borta]["Oavgjorda"] += 1
            resultat[borta][hemma]["Oavgjorda"] += 1
            
    return resultat

#get_teams_result(df)['Man City']

In [18]:
def get_home_vs_away_summary(df, home_team, away_team):

    matcher = df[(df["HomeTeam"] == home_team) & (df["AwayTeam"] == away_team)]

    resultat = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}

    for _, rad in matcher.iterrows():
        mål_hemma = rad["FTHG"]
        mål_borta = rad["FTAG"]

        if mål_hemma > mål_borta:
            resultat["Vinster"] += 1
        elif mål_hemma < mål_borta:
            resultat["Förluster"] += 1
        else:
            resultat["Oavgjorda"] += 1

    return resultat

#get_home_vs_away_summary(df,'Liverpool', 'Arsenal')

In [ ]:
def get_team_league_test():
    teams = []

    # Hantera engelska ligor (Div)
    all_eng_teams = df['HomeTeam'].astype(str).tolist() + df['AwayTeam'].astype(str).tolist()   
    unique_eng_teams = np.unique(all_eng_teams)
    teams.extend([str(team) for team in unique_eng_teams])

    # Hämta ligor från båda dataframes
    english_leagues = sorted(df['Div'].astype(str).unique().tolist())
    swedish_leagues = sorted(df_swe['League'].astype(str).unique().tolist())

    all_leagues = english_leagues + swedish_leagues
    return teams, all_leagues

get_team_league_test()

(['Arsenal',
  'Aston Villa',
  'Bournemouth',
  'Brentford',
  'Brighton',
  'Burnley',
  'Cardiff',
  'Chelsea',
  'Crystal Palace',
  'Everton',
  'Fulham',
  'Huddersfield',
  'Ipswich',
  'Leeds',
  'Leicester',
  'Liverpool',
  'Luton',
  'Man City',
  'Man United',
  'Newcastle',
  'Norwich',
  "Nott'm Forest",
  'Sheffield United',
  'Southampton',
  'Stoke',
  'Swansea',
  'Tottenham',
  'Watford',
  'West Brom',
  'West Ham',
  'Wolves'],
 ['E0', 'Allsvenskan', 'Allsvenskan '])

In [26]:
def get_team_league():
    teams_by_league = []
    all_leagues = []

    # Hantera engelska ligor
    english_leagues = sorted(df['Div'].astype(str).unique().tolist())
    for league in english_leagues:
        league_df = df[df['Div'] == league]
        teams = league_df['HomeTeam'].astype(str).tolist() + league_df['AwayTeam'].astype(str).tolist()
        unique_teams = sorted(np.unique(teams))
        teams_by_league.append(unique_teams)
        all_leagues.append(league)

    # Hantera svenska ligor, ta bort whitespaces och duplicates
    swedish_leagues_raw = df_swe['League'].astype(str).str.strip()
    swedish_leagues = sorted(swedish_leagues_raw.unique().tolist())

    for league in swedish_leagues:
        league_df = df_swe[df_swe['League'].str.strip() == league]
        teams = league_df['Home'].astype(str).tolist() + league_df['Away'].astype(str).tolist()
        unique_teams = sorted(np.unique(teams))
        teams_by_league.append(unique_teams)
        all_leagues.append(league)

    return teams_by_league, all_leagues

get_team_league()

([[np.str_('Arsenal'),
   np.str_('Aston Villa'),
   np.str_('Bournemouth'),
   np.str_('Brentford'),
   np.str_('Brighton'),
   np.str_('Burnley'),
   np.str_('Cardiff'),
   np.str_('Chelsea'),
   np.str_('Crystal Palace'),
   np.str_('Everton'),
   np.str_('Fulham'),
   np.str_('Huddersfield'),
   np.str_('Ipswich'),
   np.str_('Leeds'),
   np.str_('Leicester'),
   np.str_('Liverpool'),
   np.str_('Luton'),
   np.str_('Man City'),
   np.str_('Man United'),
   np.str_('Newcastle'),
   np.str_('Norwich'),
   np.str_("Nott'm Forest"),
   np.str_('Sheffield United'),
   np.str_('Southampton'),
   np.str_('Stoke'),
   np.str_('Swansea'),
   np.str_('Tottenham'),
   np.str_('Watford'),
   np.str_('West Brom'),
   np.str_('West Ham'),
   np.str_('Wolves')],
  [np.str_('AFC Eskilstuna'),
   np.str_('AIK'),
   np.str_('Atvidabergs'),
   np.str_('Brage'),
   np.str_('Brommapojkarna'),
   np.str_('Dalkurd'),
   np.str_('Degerfors'),
   np.str_('Djurgarden'),
   np.str_('Elfsborg'),
   np.str_('

In [ ]:
col_list = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'B365H', 'B365D', 'B365A']

In [ ]:
def get_team_form(df, team, num_matches=20):

    team_matches = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)].copy()

    team_matches = team_matches.sort_values(by='Date', ascending=False).head(num_matches)

    form = []

    for _, row in team_matches.iterrows():
        is_home = row['HomeTeam'] == team
        goals_for = row['FTHG'] if is_home else row['FTAG']
        goals_against = row['FTAG'] if is_home else row['FTHG']

        if goals_for > goals_against:
            form.append('W')
        elif goals_for == goals_against:
            form.append('D')
        else:
            form.append('L')

    return form[::-1]  

#get_team_form(df, 'Tottenham')

In [ ]:
"""""
bet_providers = ['B365', 'BFE', 'BF', 'PS']

home_cols = []
draw_cols = []
away_cols = []

for bet in bet_providers:
    home_cols.append(bet + 'H')
    draw_cols.append(bet + 'D')
    away_cols.append(bet + 'A')



df['home_max_odds'] = df[home_cols].max(axis=1)
df['draw_max_odds'] = df[draw_cols].max(axis=1)
df['away_max_odds'] = df[away_cols].max(axis=1)

col_list = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'B365H', 'B365D', 'B365A', 'PSH', 'PSD', 'PSA', 'home_max_odds', 'draw_max_odds', 'away_max_odds']
df = df[col_list]
"""""

'""\nbet_providers = [\'B365\', \'BFE\', \'BF\', \'PS\']\n\nhome_cols = []\ndraw_cols = []\naway_cols = []\n\nfor bet in bet_providers:\n    home_cols.append(bet + \'H\')\n    draw_cols.append(bet + \'D\')\n    away_cols.append(bet + \'A\')\n\n\n\ndf[\'home_max_odds\'] = df[home_cols].max(axis=1)\ndf[\'draw_max_odds\'] = df[draw_cols].max(axis=1)\ndf[\'away_max_odds\'] = df[away_cols].max(axis=1)\n\ncol_list = [\'Div\', \'Date\', \'HomeTeam\', \'AwayTeam\', \'FTHG\', \'FTAG\', \'B365H\', \'B365D\', \'B365A\', \'PSH\', \'PSD\', \'PSA\', \'home_max_odds\', \'draw_max_odds\', \'away_max_odds\']\ndf = df[col_list]\n'

In [ ]:
"""""
def divide(df):
    res = 1/df
    return res


test = df['B365A']


for index, odd in enumerate(df['PSH']):
    df.at[index, 'home_probability'] = divide(odd)
for index, odd in enumerate(df['PSD']):
    df.at[index, 'draw_probability'] = divide(odd)
for index, odd in enumerate(df['PSA']):
    df.at[index, 'away_probability'] = divide(odd)

tot_prob = df['home_probability'] + df['draw_probability'] + df['away_probability']
print(tot_prob)

"""

'""\ndef divide(df):\n    res = 1/df\n    return res\n\n\ntest = df[\'B365A\']\n\n\nfor index, odd in enumerate(df[\'PSH\']):\n    df.at[index, \'home_probability\'] = divide(odd)\nfor index, odd in enumerate(df[\'PSD\']):\n    df.at[index, \'draw_probability\'] = divide(odd)\nfor index, odd in enumerate(df[\'PSA\']):\n    df.at[index, \'away_probability\'] = divide(odd)\n\ntot_prob = df[\'home_probability\'] + df[\'draw_probability\'] + df[\'away_probability\']\nprint(tot_prob)\n\n'